In [1]:
#load packages
import pandas as pd 
import numpy as np
import fuzzymatcher

In [299]:
# load data and convert dates to datetime
fw = pd.read_csv("../Datasets/FW/FW_tail_numbers(converted to min and 24-hour format).csv", parse_dates=['dates'])
fw = fw.rename({'NAME':'medical_service'}, axis=1)

# load airport data
airports = pd.read_csv("..//Datasets/Airports/BTS_Airports_LAT_LON.csv")

In [300]:
fw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12741 entries, 0 to 12740
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tail_number       12741 non-null  object        
 1   dates             12741 non-null  datetime64[ns]
 2   aircraft          12719 non-null  object        
 3   origin            12740 non-null  object        
 4   destination       12741 non-null  object        
 5   departure         12741 non-null  object        
 6   dep_time_zone     12741 non-null  object        
 7   arrival           12741 non-null  object        
 8   arr_time_zone     12741 non-null  object        
 9   duration          12741 non-null  object        
 10  duration_minutes  12741 non-null  int64         
 11  medical_service   12741 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 1.2+ MB


In [301]:
# split origin states
fw['origin_state'] = fw.origin.str.split(", ", n = 1, expand = True)[1]
fw.origin = fw.origin.str.split(", ", n = 1, expand = True)[0]
fw = fw.iloc[:,:4].join([fw.iloc[:,-1:],fw.iloc[:,4:-1]])

# split destination states
fw['destination_state'] = fw.destination.str.split(", ", n = 1, expand = True)[1]
fw.destination = fw.destination.str.split(", ", n = 1, expand = True)[0]
fw = fw.iloc[:,:6].join([fw.iloc[:,-1], fw.iloc[:,6:-1]])

In [302]:
fw.head()

,tail_number,dates,aircraft,origin,origin_state,destination,destination_state,departure,dep_time_zone,arrival,arr_time_zone,duration,duration_minutes,medical_service
0,N7025P,2020-12-24,NaN,Red Bluff,CA,Emigrant Gap,CA,9:25,PST,10:02,PST,0:36:00,36,A PRECIOUS LIFE FLIGHT LLC ...
1,N7025P,2020-11-27,NaN,Redding Muni,None,Chico,CA,11:26,PST,11:42,PST,0:16:00,16,A PRECIOUS LIFE FLIGHT LLC ...
2,N269GJ,2020-12-18,LJ60,Aurora Muni,None,Fort Lauderdale Exec,None,8:53,CST,12:20,EST,2:26:00,146,AEROCARE MEDICAL TRANSPORT SYSTEM INC ...
3,N888CP,2021-02-11,LJ31,Aurora Muni,None,Aurora Muni,None,10:32,CST,11:25,CST,0:53:00,53,AEROCARE MEDICAL TRANSPORT SYSTEMS INC ...
4,N888CP,2021-01-24,LJ31,Akron-Canton Rgnl,None,Aurora Muni,None,16:41,EST,16:58,CST,1:17:00,77,AEROCARE MEDICAL TRANSPORT SYSTEMS INC ...


In [304]:
# replace abbreviations to full name for the best matching
fw = fw.replace(regex={r'Muni': 'Municipal', r'Rgnl':'Regional', r'Intl':'International', r"Int'l":'International',
                 r'Trml':'Terminal', r'Fld':'Field'})

In [306]:
# add origin airport information
fw = fuzzymatcher.fuzzy_left_join(fw, airports.add_suffix('_origin'), left_on = ['origin', 'origin_state'], 
                                  right_on = ['AIRPORT_NAME_origin','CITY_NAME_origin','STATE_CODE_origin']).iloc[:,3:]

# add destination airport information
fw = fuzzymatcher.fuzzy_left_join(fw, airports.add_suffix('_destination'), left_on = ['destination', 'destination_state'], 
                                  right_on = ['AIRPORT_NAME_destination','CITY_NAME_destination', 'STATE_CODE_destination']).iloc[:,3:]

fw.head()

,tail_number,dates,aircraft,origin,origin_state,destination,destination_state,departure,dep_time_zone,arrival,...,LATITUDE_origin,LONGITUDE_origin,AIRPORT_CODE_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,STATE_CODE_destination,LATITUDE_destination,LONGITUDE_destination
0,N7025P,2020-12-24,NaN,Red Bluff,CA,Emigrant Gap,CA,9:25,PST,10:02,...,40.151389,-122.251111,GAT,Gap Tallard,Gap - France,France,NaN,NaN,44.453611,6.036667
68,N7025P,2020-11-27,NaN,Redding Municipal,None,Chico,CA,11:26,PST,11:42,...,40.508333,-122.292222,CIC,Chico Municipal,Chico - CA,United States,California,CA,39.795833,-121.856667
70,N269GJ,2020-12-18,LJ60,Aurora Municipal,None,Fort Lauderdale Exec,None,8:53,CST,12:20,...,41.770833,-88.472778,FXE,Fort Lauderdale Executive,Fort Lauderdale - FL,United States,Florida,FL,26.196944,-80.170833
123,N888CP,2021-02-11,LJ31,Aurora Municipal,None,Aurora Municipal,None,10:32,CST,11:25,...,41.770833,-88.472778,AUZ,Aurora Municipal,Aurora - IL,United States,Illinois,IL,41.770833,-88.472778
126,N888CP,2021-01-24,LJ31,Akron-Canton Regional,None,Aurora Municipal,None,16:41,EST,16:58,...,40.913889,-81.441667,AUZ,Aurora Municipal,Aurora - IL,United States,Illinois,IL,41.770833,-88.472778


In [309]:
# check matching by existing states
fw['origin_matching'] = np.where((fw.origin_state.notnull()) & (fw.STATE_CODE_origin.isna()), "don't match", "match")
fw['destination_matching'] = np.where((fw.destination_state.notnull()) & (fw.STATE_CODE_destination.isna()), "don't match", "match")

In [312]:
#fw.to_csv('..//Datasets/FW/FW_with_airports.csv', index=False)

In [308]:
fw

,tail_number,dates,aircraft,origin,origin_state,destination,destination_state,departure,dep_time_zone,arrival,...,LATITUDE_origin,LONGITUDE_origin,AIRPORT_CODE_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,STATE_CODE_destination,LATITUDE_destination,LONGITUDE_destination
0,N7025P,2020-12-24,NaN,Red Bluff,CA,Emigrant Gap,CA,9:25,PST,10:02,...,40.151389,-122.251111,GAT,Gap Tallard,Gap - France,France,NaN,NaN,44.453611,6.036667
68,N7025P,2020-11-27,NaN,Redding Municipal,None,Chico,CA,11:26,PST,11:42,...,40.508333,-122.292222,CIC,Chico Municipal,Chico - CA,United States,California,CA,39.795833,-121.856667
70,N269GJ,2020-12-18,LJ60,Aurora Municipal,None,Fort Lauderdale Exec,None,8:53,CST,12:20,...,41.770833,-88.472778,FXE,Fort Lauderdale Executive,Fort Lauderdale - FL,United States,Florida,FL,26.196944,-80.170833
123,N888CP,2021-02-11,LJ31,Aurora Municipal,None,Aurora Municipal,None,10:32,CST,11:25,...,41.770833,-88.472778,AUZ,Aurora Municipal,Aurora - IL,United States,Illinois,IL,41.770833,-88.472778
126,N888CP,2021-01-24,LJ31,Akron-Canton Regional,None,Aurora Municipal,None,16:41,EST,16:58,...,40.913889,-81.441667,AUZ,Aurora Municipal,Aurora - IL,United States,Illinois,IL,41.770833,-88.472778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147697,N102WK,2021-02-12,BE9T,Smyrna,None,Shreveport Regional,None,11:19,CST,13:34,...,36.008889,-86.520000,SHV,Shreveport Regional,Shreveport - LA,United States,Louisiana,LA,32.446944,-93.825556
147699,N102WK,2021-01-28,BE9T,Shreveport Regional,None,Smyrna,None,12:30,CST,14:25,...,32.446944,-93.825556,MQY,Smyrna Airport,Smyrna - TN,United States,Tennessee,TN,36.008889,-86.520000
147702,N102WK,2021-01-05,BE9T,Shreveport Regional,None,Shreveport Regional,None,9:24,CST,10:19,...,32.446944,-93.825556,SHV,Shreveport Regional,Shreveport - LA,United States,Louisiana,LA,32.446944,-93.825556
147704,N102WK,2020-11-05,BE9T,Baton Rouge Metro,None,Shreveport Regional,None,11:20,CST,12:13,...,39.908889,-105.117222,SHV,Shreveport Regional,Shreveport - LA,United States,Louisiana,LA,32.446944,-93.825556
